In [1]:
import numpy as np
import pandas as pd
import re
import praw

In [2]:
from secrets import *

reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=CLIENT_SECRET,
                     user_agent=USER_AGENT)